In [ ]:
import torch
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import random
import torchvision
from typing import Any
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from sklearn.metrics import accuracy_score

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
writer = SummaryWriter("logs_classification")
torch.manual_seed(42)


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, list_root_dir: list, transform=None) -> None:
        super().__init__()
        self.list_root_dir = list_root_dir
        self.transform = transform
        
        self.image_paths = []
        for root_dir in list_root_dir:
            self.image_paths.extend(glob.glob(os.path.join(root_dir,"Bald","*.jpg")))
            self.image_paths.extend(glob.glob(os.path.join(root_dir,"NotBald","*.jpg")))
        # print(f"Number of images found: {len(self.image_paths)}")  # Print number of images for debugging
        # print("Sample Image Paths:")
        # print(self.image_paths[:10])
        # print("List of Root Directories:")
        # print(self.list_root_dir)
        random.shuffle(self.image_paths)

    def __len__(self) -> int:
        return len(self.image_paths)

    def __getitem__(self, index) -> Any:
            image_path = self.image_paths[index]
            
            label = 1 if image_path.split(os.sep)[-2] == "Bald" else 0
            print(f"Image Path: {image_path}, Label: {label}")  # Print image path and label for inspection
            image = Image.open(image_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
                
            return [image, label]

In [ ]:
mydataset = ImageDataset(["C:\HairLossDetection\Dataset2"])


In [ ]:
def train_model(model, loss_function, optimizer, train_loader, epoch=1):
    model.train()
    running_loss = 0.0
    total_samples = 0
    y_true = []
    y_pred = []
    for data in train_loader:     
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # Clear gradient
        optimizer.zero_grad()
        # Calculate logits
        outputs = model(images)
        # Calculate loss
        loss = loss_function(outputs, labels)
        # Calculate gradient from loss
        loss.backward()
        # Update weight
        optimizer.step()

        # Calculate loss
        running_loss += loss.item() * images.size(0)
        # Calculate total sample in data_loader
        total_samples += images.size(0)
        # Calculte y_predict for evaluation
        predicted = torch.argmax(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.detach().cpu().numpy())
    loss = running_loss/total_samples
    accuracy = accuracy_score(y_true, y_pred)
    writer.add_scalar("train/loss", loss, epoch)
    writer.add_scalar("train/accuracy", accuracy, epoch)
    return loss, accuracy

In [ ]:
def eval_model(model, loss_function, test_dataloader, epoch):
    model.eval()
    total_samples = 0
    total_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Calculate logits
            outputs = model(images)
            
            # Calculate loss of outputs and y_true
            loss = loss_function(outputs, labels)
            total_loss += loss.item() * images.size(0)
            # Calculate total sample
            total_samples += images.size(0)
            
            # Calculte y_predict for evaluation
            predicted = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.detach().cpu().numpy())
    loss = total_loss/total_samples
    accuracy = accuracy_score(y_true, y_pred)
    writer.add_scalar("test/loss", loss, epoch)
    writer.add_scalar("test/accuracy", accuracy, epoch)
    return loss, accuracy

In [ ]:
model = torchvision.models.resnet34(num_classes=2)
model = model.to(device)
print(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
list_root_dir = []
list_root_dir.append(os.path.join(os.getcwd(), "Dataset2"))

transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop([96,96]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])
epoch = 20

dataset = ImageDataset(list_root_dir, transform)

# Perform train-test split
train_dataset, test_dataset = random_split(dataset, [0.8,0.2])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

for i in range(50):
    train_loss, train_acc = train_model(model, loss_function, optimizer, train_loader, i)
    test_loss, test_acc = eval_model(model, loss_function, test_loader, i)
    print(f'''epoch {i}: train_loss {round(train_loss,4)}, train_acc {round(train_acc,4)}, test_loss {test_loss}, test_acc {round(test_acc,4)}''')
writer.flush()
writer.close()

In [ ]:
file_path = "classification.pth"
torch.save(model.state_dict(), file_path)